# mask_rle to poly_gon

In [ ]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def connect_coordinates(coordinate_list):
    # 입력된 좌표 리스트를 x 값으로 정렬합니다.
    sorted_coordinates = sorted(coordinate_list)

    connected_coordinates = []  # 연결된 좌표들을 저장할 리스트
    current_sequence = []  # 현재 연결된 좌표들을 저장할 리스트

    for i in range(len(sorted_coordinates) - 1):
        current_sequence.append(sorted_coordinates[i])

        # 현재 좌표와 다음 좌표의 x, y 값 차이를 계산합니다.
        x_diff = abs(sorted_coordinates[i][0] - sorted_coordinates[i + 1][0])
        y_diff = abs(sorted_coordinates[i][1] - sorted_coordinates[i + 1][1])

        # x 값 차이와 y 값 차이가 모두 1이 아닌 경우, 현재 연결된 좌표 시퀀스를 마무리하고 새로운 시퀀스를 시작합니다.
        if x_diff >= 2 and y_diff >= 2:
            connected_coordinates.append(current_sequence)
            current_sequence = []

    # 마지막 좌표까지 시퀀스에 추가합니다.
    current_sequence.append(sorted_coordinates[-1])
    connected_coordinates.append(current_sequence)

    return connected_coordinates

import torch
import time

start_time = time.time()

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

content = "0"
for k in range(1):
    answer = rle_decode(df.iloc[k, 2], (1024, 1024))
    answer.transpose()

    convert_answer = torch.zeros((1024, 1024), device=device)
    for i in range(1024):
        for j in range(1024):
            if i >= 1023 or i <= 0 or j >= 1023 or j <= 0:
                continue
            elif answer[i][j] == 1:
                convert_answer[i][j] = 1
            if answer[i][j] == 1 and (answer[i + 1, j] == 1 and answer[i - 1, j] == 1 and answer[i, j + 1] == 1 and
                                      answer[i, j - 1] == 1):
                convert_answer[i][j] = 0

    answer_list = []
    for i in range(1024):
        for j in range(1024):
            if convert_answer[i][j] == 1:
                answer_list.append((float(j), float(i)))

    temp = connect_coordinates(answer_list)

    file_prefix = "TRAIN_"
    file_path = "/home/hawk/decs_jupyter_lab/dataset/converted/"
    file_name = f"{file_prefix}{k:04}.txt"
    file_path += file_name

    file = open(file_path, 'w')
    for i in range(len(temp)):
        if len(temp[i]) < 3:
            continue
        file.write(content)
        file.write(" ")
        for j in range(len(temp[i])):
            file.write(str(temp[i][j][0] / float(1024)))
            file.write(" ")
            file.write(str(temp[i][j][1] / float(1024)))
            file.write(" ")
        file.write("\n")
end_time = time.time()
execution_time = end_time - start_time





# polygon sorting

In [ ]:

def find_closest_point(target_point, points_array):
    min_distance = float('inf')
    closest_point = None

    for point in points_array:
        distance = np.linalg.norm(np.array(target_point) - np.array(point))
        if distance < min_distance:
            min_distance = distance
            closest_point = point

    return closest_point

def create_new_list(numpy_array):
    new_list = [numpy_array[0].tolist()]
    remaining_points = numpy_array[1:].tolist()

    while remaining_points:
        last_point = new_list[-1]
        closest_point = find_closest_point(last_point, remaining_points)

        if closest_point:
            new_list.append(closest_point)
            remaining_points.remove(closest_point)

    return new_list

for k in range(7140) :
    file_prefix = "TRAIN_"
    file_path = "/home/hawk/decs_jupyter_lab/dataset/converted_4/"
    file_name = f"{file_prefix}{k:04}.txt"
    file_path += file_name
    new_file = "/home/hawk/decs_jupyter_lab/dataset/converted_2/"
    new_file += file_name
    with open(new_file, 'r') as ee:

        with open(file_path,'w') as dd:
            lines = ee.readlines()
            for line in lines:
                text = line.split()
                text2 = []
                string = "0 "
                for item in text :
                    text2.append(float(item))
                text3 = text2[1::]
                a = np.array(text3)
                b = a.reshape(-1,2)
                starting_x = text3[0]
                starting_y = text3[1]
                text4 = create_new_list(b)
                string += str(starting_x)
                string += " "
                string += str(starting_y)
                for i in range(len(text4) - 2) :
                    vector_x1 = text4[i+1][0] - text4[i][0]
                    vector_y1 = text4[i+1][1] - text4[i][1]
                    vector_x2 = text4[i+2][0] - text4[i+1][0]
                    vector_y2 = text4[i+2][1] - text4[i+1][1]
                    if (vector_x1 == vector_x2) or (vector_y1 == vector_y2) :
                        string += " "
                        string += str(text4[i+1][0])
                        string += " "
                        string += str(text4[i+1][1])
                string += " \n"
                if len(string) < 60 :
                    continue
                dd.write(string)

# agumentation

In [ ]:

from PIL import Image
import cv2
import albumentations as A
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob
import os
# x축 대칭 변형
transform_1 = A.Compose([
    A.VerticalFlip(p=1)
])

# y축 대칭 변형
transform_2 = A.Compose([
    A.HorizontalFlip(p=1)
])

# transpose
transform_3 = A.Compose([
    A.transpose
])





input_folder = "/home/hawk/decs_jupyter_lab/dataset/originaldata/train_img" # 원본 이미지 폴더 경로
output_folder = "/home/hawk/decs_jupyter_lab/dataset/trans_train_2" # 변형 이미지 폴더 경로


# input_folder 안의 모든 이미지 파일을 가져와서 변형을 적용하고 저장합니다.
for filename in os.listdir(input_folder):
    if filename.endswith(".png"):
        image = cv2.imread(os.path.join(input_folder, filename))

        transformed = transform_1(image=image)
        transformed_image = transformed["image"]

        output_path = os.path.join(output_folder, "transpose_" + filename)
        cv2.imwrite(output_path, transformed_image)
def swap_coordinates(line):
    coordinates = line.split()  # 공백을 기준으로 좌표들을 분리
    # x와 y 좌표를 서로 바꾸기
    swapped_coordinates = ' '.join([str(1 - float(coordinates[i])) if i % 2 == 1 else coordinates[i] for i in range(len(coordinates))])
    return swapped_coordinates

